# StyleTTS2 データセット準備

JVS002話者の音声データを用いて、3つのデータセット(phone_min4, feat_top10, full100)をStyleTTS2形式で準備します。

## データセット定義

- **phone_min4**: 4文 (VOICEACTRESS100_054, 011, 022, 015)
- **feat_top10**: 10文 (VOICEACTRESS100_006, 033, 011, 025, 045, 013, 014, 004, 017, 052)
- **full100**: 100文 (VOICEACTRESS100_001 ~ 100 全て)

## 出力

各データセットごとに以下を作成:
- `StyleTTS2/Data/jvs002_{dataset_name}/wavs/` - 音声ファイル
- `StyleTTS2/Data/jvs002_{dataset_name}/train_list.txt` - 訓練データリスト
- `StyleTTS2/Data/jvs002_{dataset_name}/val_list.txt` - 検証データリスト(10%)


In [1]:
import sys
import json
import shutil
from pathlib import Path
from typing import List, Dict
import random

# プロジェクトルートを取得
# プロジェクトルートを取得
# notebooks/styletts2/ から実行される場合: 2階層上がる
# notebooks/ から実行される場合: 1階層上がる
current_dir = Path.cwd()
if current_dir.name == "styletts2":
    PROJECT_ROOT = current_dir.parent.parent  # notebooks/styletts2 -> notebooks -> project_root
elif current_dir.name == "notebooks":
    PROJECT_ROOT = current_dir.parent  # notebooks -> project_root
else:
    PROJECT_ROOT = current_dir  # プロジェクトルートから直接実行
DATA_ROOT = PROJECT_ROOT / "data" / "jvs_ver1" / "jvs_ver1"
OUTPUT_ROOT = PROJECT_ROOT / "outputs"
STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"

# StyleTTS2をパスに追加
if STYLETTS2_DIR.exists() and str(STYLETTS2_DIR) not in sys.path:
    sys.path.insert(0, str(STYLETTS2_DIR))

SPEAKER_ID = "jvs002"

print("="*60)
print("データセット準備開始")
print("="*60)
print(f"\nプロジェクトルート: {PROJECT_ROOT}")
print(f"データルート: {DATA_ROOT}")
print(f"StyleTTS2ディレクトリ: {STYLETTS2_DIR}")
print(f"話者ID: {SPEAKER_ID}")

# 入力パスの確認
wav_dir = DATA_ROOT / SPEAKER_ID / "parallel100" / "wav24kHz16bit"
transcript_path = DATA_ROOT / SPEAKER_ID / "parallel100" / "transcripts_utf8.txt"

print(f"\n音声ファイルディレクトリ: {wav_dir} ({'存在' if wav_dir.exists() else '不存在'})")
print(f"転写テキストファイル: {transcript_path} ({'存在' if transcript_path.exists() else '不存在'})")


データセット準備開始

プロジェクトルート: /mnt/c/dev/minimal-feature-corpus-tts
データルート: /mnt/c/dev/minimal-feature-corpus-tts/data/jvs_ver1/jvs_ver1
StyleTTS2ディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2
話者ID: jvs002

音声ファイルディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/data/jvs_ver1/jvs_ver1/jvs002/parallel100/wav24kHz16bit (存在)
転写テキストファイル: /mnt/c/dev/minimal-feature-corpus-tts/data/jvs_ver1/jvs_ver1/jvs002/parallel100/transcripts_utf8.txt (存在)


In [2]:
# データセット定義の読み込み
print("\n" + "-"*60)
print("データセット定義の読み込み")
print("-"*60)

# phone_min4の読み込み
corpus_summary_path = OUTPUT_ROOT / "corpus_analysis" / SPEAKER_ID / "summary.json"
with open(corpus_summary_path, 'r', encoding='utf-8') as f:
    corpus_summary = json.load(f)
phone_min4_ids = corpus_summary['selected_ids']
print(f"phone_min4: {len(phone_min4_ids)}文")
for i, sid in enumerate(phone_min4_ids, 1):
    print(f"  {i}. {sid}")

# feat_top10の読み込み
feat_summary_path = OUTPUT_ROOT / "feature_density" / SPEAKER_ID / "summary.json"
with open(feat_summary_path, 'r', encoding='utf-8') as f:
    feat_summary = json.load(f)
feat_top10_ids = feat_summary['selected_ids']
print(f"\nfeat_top10: {len(feat_top10_ids)}文")
for i, sid in enumerate(feat_top10_ids, 1):
    print(f"  {i}. {sid}")

# full100の読み込み
all_ids = []
with open(transcript_path, 'r', encoding='utf-8') as f:
    for line in f:
        if ':' in line:
            file_id = line.strip().split(':')[0]
            all_ids.append(file_id)
full100_ids = sorted(all_ids)
print(f"\nfull100: {len(full100_ids)}文 (全{len(full100_ids)}文)")

# データセット定義をまとめる
datasets = {
    "phone_min4": phone_min4_ids,
    "feat_top10": feat_top10_ids,
    "full100": full100_ids
}



------------------------------------------------------------
データセット定義の読み込み
------------------------------------------------------------
phone_min4: 4文
  1. VOICEACTRESS100_054
  2. VOICEACTRESS100_011
  3. VOICEACTRESS100_022
  4. VOICEACTRESS100_015

feat_top10: 10文
  1. VOICEACTRESS100_006
  2. VOICEACTRESS100_033
  3. VOICEACTRESS100_011
  4. VOICEACTRESS100_025
  5. VOICEACTRESS100_045
  6. VOICEACTRESS100_013
  7. VOICEACTRESS100_014
  8. VOICEACTRESS100_004
  9. VOICEACTRESS100_017
  10. VOICEACTRESS100_052

full100: 100文 (全100文)


In [3]:
# 転写テキストの読み込み
print("\n" + "-"*60)
print("転写テキストの読み込み")
print("-"*60)

transcripts = {}
with open(transcript_path, 'r', encoding='utf-8') as f:
    for line in f:
        if ':' in line:
            file_id, text = line.strip().split(':', 1)
            transcripts[file_id] = text.strip()

print(f"転写テキスト数: {len(transcripts)}")
print(f"\nサンプル:")
for i, (fid, text) in enumerate(list(transcripts.items())[:3], 1):
    print(f"  {i}. {fid}: {text[:50]}...")



------------------------------------------------------------
転写テキストの読み込み
------------------------------------------------------------
転写テキスト数: 100

サンプル:
  1. VOICEACTRESS100_001: また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配されることも多い。...
  2. VOICEACTRESS100_002: ニューイングランド風は、牛乳をベースとした、白いクリームスープであり、ボストンクラムチャウダーとも呼...
  3. VOICEACTRESS100_003: コンピュータゲームのメーカーや、業界団体などに関連する人物のカテゴリ。...


In [4]:
# 音素化関数の定義
print("\n" + "-"*60)
print("音素化関数の定義")
print("-"*60)

try:
    from phonemizer import phonemize
    
    def phonemize_japanese(text: str) -> str:
        """
        日本語テキストをeSpeak-NGで音素化
        
        Args:
            text: 日本語テキスト
            
        Returns:
            音素化されたテキスト
        """
        try:
            phonemized = phonemize(
                text,
                backend='espeak',
                language='ja',
                strip=True,
                preserve_punctuation=True,
                with_stress=False,
                njobs=1
            )
            return phonemized.strip()
        except Exception as e:
            print(f"音素化エラー (テキスト: {text[:50]}...): {e}")
            return text  # エラー時は元のテキストを返す
    
    # テスト
    test_text = "こんにちは、今日は良い天気ですね。"
    test_phonemized = phonemize_japanese(test_text)
    print(f"テスト:")
    print(f"  元のテキスト: {test_text}")
    print(f"  音素化結果: {test_phonemized}")
    print("✓ 音素化関数が正常に動作しています")
    
except ImportError:
    print("✗ phonemizerがインストールされていません")
    print("pip install phonemizer を実行してください")
    raise
except Exception as e:
    print(f"✗ 音素化テストに失敗: {e}")
    import traceback
    traceback.print_exc()



------------------------------------------------------------
音素化関数の定義
------------------------------------------------------------
テスト:
  元のテキスト: こんにちは、今日は良い天気ですね。
  音素化結果: ko̞nnitɕihä (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə hä (en)tʃaɪniːz(ja)le̞tə i (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə te̞ (en)dʒapəniːz(ja)le̞tə sɯᵝ ne̞
✓ 音素化関数が正常に動作しています


In [5]:
# データセット準備関数
def prepare_dataset(dataset_name: str, file_ids: List[str], val_ratio: float = 0.1):
    """
    StyleTTS2用のデータセットを準備
    
    Args:
        dataset_name: データセット名
        file_ids: ファイルIDのリスト
        val_ratio: 検証データの割合
    """
    print(f"\n{'='*60}")
    print(f"データセット準備: {dataset_name} ({len(file_ids)}文)")
    print(f"{'='*60}")
    
    # 出力ディレクトリの作成
    output_dir = STYLETTS2_DIR / "Data" / f"jvs002_{dataset_name}"
    wavs_dir = output_dir / "wavs"
    wavs_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"出力ディレクトリ: {output_dir}")
    
    # ファイルIDをシャッフル（検証データ分割のため）
    file_ids_shuffled = file_ids.copy()
    random.seed(42)  # 再現性のため
    random.shuffle(file_ids_shuffled)
    
    # 訓練/検証データの分割
    n_val = max(1, int(len(file_ids_shuffled) * val_ratio))
    val_ids = file_ids_shuffled[:n_val]
    train_ids = file_ids_shuffled[n_val:]
    
    print(f"訓練データ: {len(train_ids)}文, 検証データ: {len(val_ids)}文")
    
    # 音声ファイルのコピーとリストの作成
    train_list = []
    val_list = []
    
    for file_id in file_ids:
        # 元の音声ファイルパス
        src_wav = wav_dir / f"{file_id}.wav"
        
        if not src_wav.exists():
            print(f"⚠️ 警告: 音声ファイルが見つかりません: {src_wav}")
            continue
        
        # 転写テキストの取得
        if file_id not in transcripts:
            print(f"⚠️ 警告: 転写テキストが見つかりません: {file_id}")
            continue
        
        text = transcripts[file_id]
        
        # 音素化
        try:
            phonemized_text = phonemize_japanese(text)
        except Exception as e:
            print(f"⚠️ 警告: 音素化に失敗 ({file_id}): {e}")
            continue
        
        # 音声ファイルをコピー
        dst_wav = wavs_dir / f"{file_id}.wav"
        shutil.copy2(src_wav, dst_wav)
        
        # リストエントリの作成 (形式: wavs/FILENAME.wav|phonemized_text|0)
        list_entry = f"wavs/{file_id}.wav|{phonemized_text}|0"
        
        if file_id in val_ids:
            val_list.append(list_entry)
        else:
            train_list.append(list_entry)
    
    # リストファイルの保存
    train_list_path = output_dir / "train_list.txt"
    val_list_path = output_dir / "val_list.txt"
    
    with open(train_list_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(train_list))
    
    with open(val_list_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(val_list))
    
    print(f"\n✓ データセット準備完了:")
    print(f"  音声ファイル: {wavs_dir} ({len(file_ids)}ファイル)")
    print(f"  訓練リスト: {train_list_path} ({len(train_list)}エントリ)")
    print(f"  検証リスト: {val_list_path} ({len(val_list)}エントリ)")
    
    return output_dir

print("✓ データセット準備関数を定義しました")


✓ データセット準備関数を定義しました


In [6]:
# 各データセットの準備
prepared_datasets = {}

for dataset_name, file_ids in datasets.items():
    try:
        output_dir = prepare_dataset(dataset_name, file_ids)
        prepared_datasets[dataset_name] = output_dir
    except Exception as e:
        print(f"✗ エラー: {dataset_name} の準備に失敗: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "="*60)
print("全データセット準備完了")
print("="*60)
for name, path in prepared_datasets.items():
    print(f"  {name}: {path}")



データセット準備: phone_min4 (4文)
出力ディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4
訓練データ: 3文, 検証データ: 1文

✓ データセット準備完了:
  音声ファイル: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4/wavs (4ファイル)
  訓練リスト: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4/train_list.txt (3エントリ)
  検証リスト: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4/val_list.txt (1エントリ)

データセット準備: feat_top10 (10文)
出力ディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_feat_top10
訓練データ: 9文, 検証データ: 1文

✓ データセット準備完了:
  音声ファイル: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_feat_top10/wavs (10ファイル)
  訓練リスト: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_feat_top10/train_list.txt (9エントリ)
  検証リスト: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_feat_top10/val_list.txt (1エントリ)

データセット準備: full100 (100文)
出力ディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_full100
訓練データ: 90文

In [7]:
# 作成されたリストファイルの確認
print("\n" + "-"*60)
print("作成されたリストファイルの確認")
print("-"*60)

for dataset_name in datasets.keys():
    dataset_dir = STYLETTS2_DIR / "Data" / f"jvs002_{dataset_name}"
    train_list_path = dataset_dir / "train_list.txt"
    val_list_path = dataset_dir / "val_list.txt"
    
    if train_list_path.exists() and val_list_path.exists():
        with open(train_list_path, 'r', encoding='utf-8') as f:
            train_lines = f.readlines()
        with open(val_list_path, 'r', encoding='utf-8') as f:
            val_lines = f.readlines()
        
        print(f"\n{dataset_name}:")
        print(f"  訓練データ: {len(train_lines)}エントリ")
        print(f"  検証データ: {len(val_lines)}エントリ")
        
        if train_lines:
            print(f"\n  訓練データサンプル (最初の1行):")
            print(f"    {train_lines[0].strip()[:100]}...")
        if val_lines:
            print(f"\n  検証データサンプル (最初の1行):")
            print(f"    {val_lines[0].strip()[:100]}...")
    else:
        print(f"\n{dataset_name}: ファイルが見つかりません")



------------------------------------------------------------
作成されたリストファイルの確認
------------------------------------------------------------

phone_min4:
  訓練データ: 3エントリ
  検証データ: 1エントリ

  訓練データサンプル (最初の1行):
    wavs/VOICEACTRESS100_054.wav|(en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə nä ni ɯᵝ (en)dʒapəniːz(ja)l...

  検証データサンプル (最初の1行):
    wavs/VOICEACTRESS100_022.wav|ko̞no̞to̞kini (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(...

feat_top10:
  訓練データ: 9エントリ
  検証データ: 1エントリ

  訓練データサンプル (最初の1行):
    wavs/VOICEACTRESS100_006.wav|tsɯᵝ ɯᵝ ɽe̞ ɴ hä ɽɯᵝ to̞ (en)tʃaɪniːz(ja)le̞tə hä ɯᵝ (en)dʒapəniːz(ja)l...

  検証データサンプル (最初の1行):
    wavs/VOICEACTRESS100_004.wav|sä (en)dʒapəniːz(ja)le̞tə çi (en)dʒapəniːz(ja)le̞tə sɯᵝ mä ne̞ (en)dʒap...

full100:
  訓練データ: 90エントリ
  検証データ: 10エントリ

  訓練データサンプル (最初の1行):
    wavs/VOICEACTRESS100_001.wav|mätä (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə no̞ jo̞ ɯᵝ ni (en)tʃaɪ...

  検証データサンプル (最初の1行):
    wavs/VOICEACTRESS100_002.wav|ni ɯᵝ (en)dʒapəniːz(ja)le̞tə

## データセット準備完了

各データセットが `StyleTTS2/Data/jvs002_{dataset_name}/` に準備されました。

次のステップ:
- `02_generate_configs.ipynb` で設定ファイルを生成
